In [1]:
import pandas as pd
import glob
import os

# 1. מציאת הקובץ האחרון שנוצר 
# נחפש את כל הקבצים המתחילים ב-'synthetic_logs_' בתיקייה הנוכחית
list_of_files = glob.glob('synthetic_logs_*.csv') 
# נבחר את הקובץ שנוצר אחרון
latest_file = max(list_of_files, key=os.path.getctime)

# 2. טעינת הקובץ ל-DataFrame
try:
    df = pd.read_csv(latest_file)
    print(f"\n--- Successfully loaded file: {latest_file} ---")
    
    # 3. הצגת חמש השורות הראשונות
    print("\n--- First 5 Rows of Generated Data ---")
    print(df.head())
    
    # 4. הצגת מידע על הנתונים (כדי לראות ערכי NaN)
    print("\n--- DataFrame Info (Check for NaN) ---")
    df.info()

except Exception as e:
    print(f"Error loading CSV file: {e}")


--- Successfully loaded file: synthetic_logs_20251120_230757.csv ---

--- First 5 Rows of Generated Data ---
               timesamp source_ip http_method  \
0  2021-07-16T13:57:49Z   1.2.3.4         GET   
1  2021-07-16T13:57:49Z   1.2.3.4         GET   
2  2021-07-16T13:57:49Z   1.2.3.4         GET   
3  2021-07-16T13:57:49Z   1.2.3.4         GET   
4  2021-07-16T13:57:49Z   1.2.3.4         GET   

                                       requested_url           query_params  \
0                               /api/v1/product/list  &sortby=id,name,price   
1          /api/v1/product/list?sortby=id,name,price                    NaN   
2          /api/v1/product/list?sortby=id,name,price                    NaN   
3          /api/v1/product/list?sortby=id,name,price                    NaN   
4  /api/v1/product/list?sortby=id,name,price&is_m...                    NaN   

  is_malicious  
0          NaN  
1         True  
2         True  
3         True  
4         True  

--- DataFrame Inf

In [2]:
# Assuming df is the DataFrame loaded in the previous step

# --- 1. תיקון שמות העמודות (Fixing Typos) ---
print("\n--- 1. Fixing Column Typos ---")
df.rename(columns={'timesamp': 'timestamp', 
                   # ניתן להוסיף כאן תיקונים נוספים אם תתגלה טעות בעתיד
                  }, inplace=True)
print("Columns renamed successfully.")


# --- 2. טיפול בערכי NaN ובערכים חסרים ---

# א. טיפול בערכי התווית (is_malicious): 
# נמלא את הערך החסר (NaN) ב-False, ונמיר את העמודה לטיפוס בוליאני.
# הערך True מופיע כטקסט, לכן נשתמש בהמרה ל-boolean.
df['is_malicious'] = df['is_malicious'].fillna(False).astype(bool)

# ב. טיפול בערכים החסרים בפרמטרים (query_params): 
# נמלא NaN במחרוזת ריקה כדי למנוע קריסה בהמשך ניתוח הטקסט.
df['query_params'] = df['query_params'].fillna("")


# --- 3. הכנת הנתונים למודל (Feature & Target) ---

# בחירת המאפיין (הטקסט) ומשתנה המטרה (התווית)
X = df['query_params']
y = df['is_malicious']

print("\n--- Data Cleaning Summary ---")
print(f"Total non-malicious (False): {y.value_counts().get(False, 0)}")
print(f"Total malicious (True): {y.value_counts().get(True, 0)}")
print(f"Missing values after cleanup: {df.isnull().sum().sum()}")

print("\nData is ready for Model Training (Benchmark)")


--- 1. Fixing Column Typos ---
Columns renamed successfully.

--- Data Cleaning Summary ---
Total non-malicious (False): 1
Total malicious (True): 5
Missing values after cleanup: 0

Data is ready for Model Training (Benchmark)


C:\Users\itaym\AppData\Local\Temp\ipykernel_27852\1174559681.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_malicious'] = df['is_malicious'].fillna(False).astype(bool)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

# --- 1. הכנת נתונים (Target והמאפיין הטקסטואלי) ---

# X: המאפיין הטקסטואלי (הפרמטרים של הלוג)
X = df['query_params']
# y: משתנה המטרה (התווית - True/False)
y = df['is_malicious']

print("\n--- Starting Benchmark Training (Logistic Regression) ---")

# --- 2. וקטוריזציה של הטקסט (הפיכת טקסט למספרים) ---
# CountVectorizer היא השיטה הפשוטה ביותר ל-Benchmark
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)


# --- 3. חלוקה ל-Train/Test ---
# נשתמש ב-stratify כדי לוודא שחלוקת המחלקות הלא מאוזנת נשמרת.
# הערה: עקב גודל הנתונים הקטן (6 רשומות), ה-Test set יהיה קטן מאוד.
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.3, random_state=42, stratify=y
)
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


# --- 4. אימון המודל ---
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train, y_train)


# --- 5. הערכה (Evaluation) ---
y_pred = model.predict(X_test)

print("\n--- Evaluation Results ---")

# מטריצת בלבול (Confusion Matrix)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# F1-Score (מדד קריטי לנתונים לא מאוזנים)
print(f"F1 Score: {f1_score(y_test, y_pred, average='binary'):.4f}")

# Accuracy (דיוק כללי)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

print("\nBenchmark Done.")



--- Starting Benchmark Training (Logistic Regression) ---


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import numpy as np

# X, y הם הנתונים שנוקו בשלב הקודם
X = df['query_params']
y = df['is_malicious']

print("\n--- Starting K-Fold Cross-Validation Benchmark ---")

# --- 1. וקטוריזציה של הטקסט (על כל הנתונים) ---
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)


# --- 2. הגדרת Cross-Validation (שימוש ב-StratifiedKFold בגלל האיזון) ---
# נבחר K=3 קיפולים (הערה: זה המקסימום ההגיוני עם 6 רשומות)
# StratifiedKFold משמר את יחס המחלקות בכל קיפול (K-Fold)
N_SPLITS = 3
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# מערכים לאחסון תוצאות
f1_scores = []
accuracy_scores = []


# --- 3. אימון והערכה איטרטיבית ---
print(f"Running {N_SPLITS}-Fold Cross-Validation...")
i = 0
for train_index, test_index in skf.split(X_vectorized, y):
    i += 1
    
    # חלוקה ל-Train/Test עבור הקיפול הנוכחי
    X_train, X_test = X_vectorized[train_index], X_vectorized[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # אימון המודל
    model = LogisticRegression(solver='liblinear', random_state=42)
    model.fit(X_train, y_train)
    
    # חיזוי והערכה
    y_pred = model.predict(X_test)
    
    # שמירת המדדים
    f1 = f1_score(y_test, y_pred, average='binary', zero_division=0)
    acc = accuracy_score(y_test, y_pred)
    
    f1_scores.append(f1)
    accuracy_scores.append(acc)
    print(f"Fold {i} Results: Accuracy={acc:.2f}, F1-Score={f1:.2f}")


# --- 4. סיכום התוצאות ---
print("\n--- Final Benchmark Results (Macro Average) ---")
print(f"Average F1 Score (across {N_SPLITS} folds): {np.mean(f1_scores):.4f}")
print(f"Average Accuracy (across {N_SPLITS} folds): {np.mean(accuracy_scores):.4f}")

print("\nBenchmark Done.")


--- Starting K-Fold Cross-Validation Benchmark ---
Running 3-Fold Cross-Validation...


C:\Users\itaym\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.True_

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer # שימוש ב-TFIDF במקום CountVectorizer (עדיף)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import numpy as np

# X, y הם הנתונים שנוקו בשלב הקודם
X = df['query_params']
y = df['is_malicious']

print("\n--- Starting Final Benchmark with Simple Train/Test Split ---")

# --- 1. וקטוריזציה של הטקסט (TFIDF) ---
# נשתמש ב-TFIDF, שהוא חזק יותר בטיפול במונחים חשובים
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)


# --- 2. חלוקה ל-Train/Test (ללא Stratify) ---
# עקב הכשל ב-stratify, נשתמש בחלוקה פשוטה, תוך סיכון לאי-ייצוג.
# נשתמש ב-Test Size קטן מאוד כדי להשאיר דוגמאות False ב-Train.
# בגלל 1 דוגמה False, נחלק 5/1.
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.15, random_state=42
)
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


# --- 3. אימון המודל ---
model = LogisticRegression(solver='liblinear', random_state=42)
# במידה ו-y_train מכיל רק True, זה עדיין יכשל. 
# נכריח את המערכת לרוץ על הנתונים הזמינים.
try:
    model.fit(X_train, y_train)
    
    # --- 4. הערכה (Evaluation) ---
    y_pred = model.predict(X_test)
    
    print("\n--- Evaluation Results ---")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print(f"F1 Score: {f1_score(y_test, y_pred, average='binary', zero_division=0):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
except ValueError as e:
    print(f"FATAL ERROR: Could not train model. The small split size still resulted in only one class in the training set.")
    print("This confirms the synthetic dataset is TOO SMALL to run a valid benchmark.")
    
print("\nBenchmark Attempt Done.")


--- Starting Final Benchmark with Simple Train/Test Split ---
Training samples: 5, Test samples: 1
FATAL ERROR: Could not train model. The small split size still resulted in only one class in the training set.
This confirms the synthetic dataset is TOO SMALL to run a valid benchmark.

Benchmark Attempt Done.
